In [5]:
PROT_LOWER = -1
PROT_UPPER = 1

curve_u = []
curve_l = []
curves = {}
parameters = dict()

In = 'f.In'
Isc = '1000*f.Isc'
i_nf = '1.05*f.In*d.i_r'
i_f = '1.2*f.In*d.i_r'
tol_im_p = 10
tol_im_m = 10
tol_ii_p = 15
tol_ii_m = 15

# tol_tm_p = 15
# tol_tm_m = 15
tm_p = {0:0.08, 0.1:0.14, 0.2:0.2, 0.3:0.32, 0.4:0.5}
tm_m = {0:0.02, 0.1:0.08, 0.2:0.14, 0.3:0.23, 0.4:0.35}
t_i_p = 0.055
t_i_m = 0.001

tol_tr_p = 0
tol_tr_m = 20
# tol_m_i2t_p = 15
# tol_m_i2t_m = 15

i_i2t = 10
ir_i2t = 6


select_expr_list = ['d.i2t_on is False', 'd.i2t_on is True']

def get_curves(Ir, I1, Tr, T_conv, Im, Tm, Ii, Ti, i_tol, t_tol, curve_type, i2t_tol):
    k_I2T = str(i_i2t**2) + '*d.t_m'
    kr_I2t = str(ir_i2t**2) + '*d.t_r'
    curve_vars = {'tms'         : 1, 
                    'i_n'         : Ir, 
                    'i1'          : I1,
                    'i2'          : Im,
                    't_min'       : Tm, 
                    'n'           : 100,
                    'i_tol'       : i_tol,
                    't_tol'       : t_tol,
                    'curve_type'  : curve_type,
                    'k_i2t'       : kr_I2t,
                    }
    # M, I
    curve1 = [('point', I1, T_conv),
                ('I2T', curve_vars),
                ('point', Im, Tm),
                ('point', Ii, Tm),
                ('point', Ii, Ti),
                ('point', Isc, Ti),]
    # M, I, I2t
    curve2 = [('point', I1, T_conv),
                ('I2T', curve_vars),
                ('I2T', i2t_tol, Ir, Im, Ii, Tm, 100, k_I2T, 2),
                ('point', Ii, Tm),
                ('point', Ii, Ti),
                ('point', Isc, Ti),]
    return curve1, curve2

# Upper curves
Ir = 'd.i_r*' + In
I1 = i_f
Tr = 'd.t_r'
T_conv = '3600 if f.In <= 63  else 2*3600'
Im = 'd.i_m*' + Ir + '*' + str((100+tol_im_p)/100)
# Tm = 'd.t_m*' + str((100+tol_tm_p)/100)
Tm = str(tm_p) + '[d.t_m]'
Ii = 'd.i_i*' + In + '*' + str((100+tol_ii_p)/100)
Ti = t_i_p
i_tol = i_f + '*100-100'
t_tol = tol_tr_p
curve_type = PROT_UPPER
# i2t_tol = (100+tol_m_i2t_p)/100
i2t_tol = Tm + '/d.t_m if d.t_m != 0 else 1'
curve_u1, curve_u2 = get_curves(Ir, I1, Tr, T_conv, Im, Tm, Ii, Ti, i_tol, t_tol, curve_type, i2t_tol)
# Lower curves
Ir = 'd.i_r*' + In
I1 = i_nf
Tr = 'd.t_r'
T_conv = '3600 if f.In <= 63  else 2*3600'
Im = 'd.i_m*' + Ir + '*' + str((100-tol_im_m)/100)
# Tm = 'd.t_m*' + str((100-tol_tm_m)/100)
Tm = str(tm_m) + '[d.t_m]'
Ii = 'd.i_i*' + In + '*' + str((100-tol_ii_m)/100)
Ti = t_i_m
i_tol = i_nf + '*100-100'
t_tol = -tol_tr_m
curve_type = PROT_LOWER
# i2t_tol = (100-tol_m_i2t_m)/100
i2t_tol = Tm + '/d.t_m if d.t_m != 0 else 1'
curve_l1, curve_l2 = get_curves(Ir, I1, Tr, T_conv, Im, Tm, Ii, Ti, i_tol, t_tol, curve_type, i2t_tol)

parameters = {  'head_l'    : ['Long time protection', '', '', None, '', 'heading'],
                'i_r'       : ['Ir', 'xIn', 1, None, 'Long-time pickup current', [2,0.4,1]],
                't_r'       : ['tr', 's', 16, [0.5,1,2,4,8,12,16,20,24], 'Long-time tripping delay @ 6xIr'],
                'head_s'    : ['Short time protection', '', '', None, '', 'heading'],
                'i_m'       : ['Isd', 'xIr', 10, [1.5,2,2.5,3,4,5,6,8,10], 'Short-time pickup current'],
                't_m'       : ['tsd', 's', 0.4, [0,0.1,0.2,0.3,0.4], 'Short-time tripping delay (@ 10xIsd for I2T=ON)'],
                'i2t_on'        : ['I2t ON', '', False, None, '', 'bool'],
                'head_i'        : ['Instantaneous protection', '', '', None, '', 'heading'],
                'i_i'           : ['Ii', 'xIn', 11, None, 'Instantaneous pickup current', [1,1.5,15]],
                }

curves = {'select_expr_list': select_expr_list,
'curve_u1': curve_u1, 'curve_l1': curve_l1,
          'curve_u2': curve_u2, 'curve_l2': curve_l2}


In [6]:
import json, re

data = {'type'          : 'protection',
        'parameters'    : parameters,
        'data'          : curves,
        'graph_model'   : []}

with open('micrologic_l.json', 'w') as fp:
    json.dump(data, fp, indent=2)

# output = json.dumps(data, indent=2)
# output2 = re.sub(r'": \[\s+', '": [', output)
# output3 = re.sub(r'",\s+', '", ', output2)
# output4 = re.sub(r'"\s+\]', '"]', output3)

# with open('legrand_uP.json', 'w') as fp:
#     fp.write(output4)